# Use TTH_Template Custom Notebook template

In [1]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

from sklearn.model_selection import train_test_split
# Importing Scikit-learn's preprocessing utilities for encoding categorical variables and scaling numerical data
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, classification_report

pd.options.display.max_columns = 50
warnings.filterwarnings("ignore")

from fosforml.model_manager.snowflakesession import get_session
from fosforml import register_model

In [2]:
## for no font error:

# Set Matplotlib's default font family to 'DeJavu Serif' to ensure a consistent font style across plots
plt.rcParams['font.family'] = 'DeJavu Serif'

# Read data from Snowflake

In [3]:
my_session = get_session()

In [ ]:
## table_name = 'FLIGHTS'
table_name = 'FLIGHTS_FULL'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [ ]:
## FILTERING DATA for just 2 airlines

options = ['Southwest Airlines Co.', 'Delta Air Lines Inc.'] 
  
# selecting rows based on condition 
flights = df.loc[df['AIRLINE'].isin(options)] 

## check
## flights['AIRLINE'].unique()

In [ ]:
## flights = df.copy()
flights_needed_data = flights.copy()

In [ ]:
flights_needed_data.shape

## OLD VALUE: (5819079, 31)

In [ ]:
flights_needed_data.info()

In [ ]:
flights_needed_data.head

In [ ]:
## flights_needed_data = df.loc[(df['fly_date'] <= '2024-10-31')]
## flights_needed_data = flights[0:100000]  # getting a segment 

In [ ]:
def categorize_time(SCHEDULED_ARRIVAL):
    if SCHEDULED_ARRIVAL >= 500 and SCHEDULED_ARRIVAL< 800: return 'Early morning'
    elif SCHEDULED_ARRIVAL >= 800 and SCHEDULED_ARRIVAL < 1100: return 'Late morning'
    elif SCHEDULED_ARRIVAL >= 1100 and SCHEDULED_ARRIVAL < 1400: return 'Around noon'
    elif SCHEDULED_ARRIVAL >= 1400 and SCHEDULED_ARRIVAL < 1700: return 'Afternoon'
    elif SCHEDULED_ARRIVAL >= 1700 and SCHEDULED_ARRIVAL < 2000: return 'Evening'
    elif SCHEDULED_ARRIVAL >= 2000 and SCHEDULED_ARRIVAL < 2300: return 'Night'
    elif SCHEDULED_ARRIVAL >= 2300 or SCHEDULED_ARRIVAL < 200: return 'Late night'
    elif SCHEDULED_ARRIVAL >= 200 or SCHEDULED_ARRIVAL < 500: return 'Dawn'
#    else:
#        return 'Dawn'

# Apply the function to the Age column using the apply() function
flights_needed_data['ARRIVAL_TIME_SEGMENT'] = flights_needed_data['SCHEDULED_ARRIVAL'].apply(categorize_time)

In [ ]:
flights_needed_data

In [ ]:
 flights['AIRLINE__CODE'].unique()

In [ ]:
flights_needed_data.value_counts('DIVERTED')  # will tell us the no. of flights which were diverted

In [ ]:
flights_needed_data.info()

In [ ]:
# filtering out unnecessary columns
flights_needed_data=flights_needed_data.drop(['YEAR','FLIGHT_NUMBER','AIRLINE__CODE','TAIL_NUMBER','TAXI_OUT',
                                              'SCHEDULED_TIME','WHEELS_OFF','ELAPSED_TIME',
                                              'AIR_TIME','WHEELS_ON','DAY_OF_WEEK','TAXI_IN','CANCELLATION_REASON', 'DEST_LATITUDE',
                                              'DEST_LONGITUDE','ORIGIN_LATITUDE', 'ORIGIN_LONGITUDE','ORIGIN_STATE', 'ORIGIN_COUNTRY', 
                                              'DEST_AIRPORT','DEST_CITY','DEST_STATE','DEST_COUNTRY', 'ORIGIN_CITY', 'ORIGIN_AIRPORT' ],
                                             axis=1)
# REMOVED DISTANCE FROM THIS LIST

In [ ]:
## flights_needed_data=flights_needed_data.drop(['ORIGIN_AIRPORT' ],
#                                             axis=1)

In [ ]:
flights_needed_data.isnull().sum()

In [ ]:
flights_needed_data.info()

## DO NOT RUN THIS CODE
#flights_needed_data = flights_needed_data.apply(lambda x: x.fillna(x.mean()), axis=0)

cols = ['DEPARTURE_DELAY','ARRIVAL_DELAY','AIR_SYSTEM_DELAY',
        'SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']
cols1 = ['DEPARTURE_TIME']
cols2 = ['ARRIVAL_TIME']
#cols = ['DEPARTURE_TIME','DEPARTURE_DELAY','ARRIVAL_TIME','ARRIVAL_DELAY','AIR_SYSTEM_DELAY',
#        'SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']

for column in cols1:
    print (column)
    flights_needed_data[column].fillna(flights_needed_data[['SCHEDULED_DEPARTURE']]), inplace=True)

for column in cols2:
    print (column)
    flights_needed_data[column].fillna(flights_needed_data[['SCHEDULED_ARRIVAL']]), inplace=True)
    
for column in cols:
    print (column)
    flights_needed_data[column].fillna(flights_needed_data[column].mean(), inplace=True)

In [ ]:
#flights_needed_data = flights_needed_data.apply(lambda x: x.fillna(x.mean()), axis=0)

cols = ['DEPARTURE_TIME', 'ARRIVAL_TIME','DEPARTURE_DELAY','ARRIVAL_DELAY','AIR_SYSTEM_DELAY',
        'SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']

for column in cols:
    print (column)
    flights_needed_data[column].fillna(flights_needed_data[column].mean(), inplace=True)

In [ ]:
# replacing all NaN values with the mean of the attribute in which they are present
#flights_needed_data=flights_needed_data.fillna(flights_needed_data.mean())

In [ ]:
flights_needed_data.head()

In [ ]:
# creating a new column; it will tell if the flight was delayed or not
result=[]

# Create Target/Result column for Classifier

In [ ]:
for row in flights_needed_data['ARRIVAL_DELAY']:
  if row > 5:
    result.append(1)
  else:
    result.append(0) 

In [ ]:
flights_needed_data['result'] = result

In [ ]:
flights_needed_data.value_counts('result')

In [ ]:
# removing some more columns
flights_needed_data=flights_needed_data.drop(['ORIGIN_AIRPORT_CODE', 'DESTINATION_AIRPORT_CODE', 'ARRIVAL_TIME', 'ARRIVAL_DELAY'],axis=1)
flights_needed_data

In [ ]:
flights_needed_data=flights_needed_data.drop(['FLY_DATE'], axis=1)

In [ ]:
#Get list of categorical variables
s = (flights_needed_data.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables in the dataset:", object_cols)

In [ ]:
#Label Encoding the object dtypes.
LE=LabelEncoder()
for i in object_cols:
    flights_needed_data[i]=flights_needed_data[[i]].apply(LE.fit_transform)
    
print("All features are now numerical")

In [ ]:
print(flights_needed_data.max(axis=0)) # will return max value of each column
print(flights_needed_data.min(axis=0)) # will return min value of each column

# Train test split

In [ ]:
flights_needed_data.info()

In [ ]:
test = flights_needed_data[flights_needed_data['MONTH'] >= 11] 
#test=test.drop(['FLY_DATE'], axis=1)
train = flights_needed_data[flights_needed_data['MONTH'] < 11]
#train=train.drop(['FLY_DATE'], axis=1)
test_data = test.values
train_data = train.values
X_train, y_train = train_data[:,:-1], train_data[:,-1]
X_test, y_test = test_data[:,:-1], test_data[:,-1]
#train_data = flights_needed_data[flights_needed_data['FLY_DATE'].apply(lambda x:x.date()) < datetime.date(2024, 11, 1)]
#test_data  = flights_needed_data[flights_needed_data['FLY_DATE'].apply(lambda x:x.date()) > datetime.date(2024, 10, 31)]

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
X_train

In [ ]:
sc = StandardScaler()
X_train_1 = sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

# Model training and Prediction

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

In [ ]:
y_prob = clf.predict_proba(X_test)[:,1]

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
auc_score = roc_auc_score(y_test, y_pred)
auc_score

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
flights_needed_data.shape

In [ ]:
flights_needed_df = flights_needed_data.drop(['result'],axis=1)

In [ ]:
final_data = flights_needed_df.values

In [ ]:
final_data = sc.transform(final_data)

In [ ]:
y_pred = clf.predict(final_data)

In [ ]:
final_data

In [ ]:
y_prob = clf.predict_proba(final_data)[:,1]

In [ ]:
len(y_pred)

In [ ]:
clf.feature_importances_

In [ ]:
type(X_train),type(X_test),type(y_train),type(y_test),type(y_pred),type(y_prob)

In [ ]:
column = ['MONTH', 'DAY', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME','DEPARTURE_DELAY','DISTANCE','SCHEDULED_ARRIVAL', 'DIVERTED', 'CANCELLED', 
          'AIR_SYSTEM_DELAY','SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY','WEATHER_DELAY','AIRLINE', 'ARRIVAL_TIME_SEGMENT']


In [ ]:
X_train_df = pd.DataFrame(X_train, columns=column)
X_test_df = pd.DataFrame(X_test, columns=column)
y_train_df = pd.DataFrame(y_train, columns=['ACTUAL_DELAY'])
y_test_df = pd.DataFrame(y_test, columns=['ACTUAL_DELAY'])
y_pred_df = pd.DataFrame(y_pred, columns=['PREDICTED_DELAY'])
y_prob_df = pd.DataFrame(y_prob, columns=['PROBABILITY'])

# Model Registration

In [ ]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=clf, 
    session=my_session,
    x_train=X_train_df,
    y_train=y_train_df,
    x_test=X_test_df,
    y_test=y_test_df,
    y_pred=y_pred_df,
    y_prob=y_prob_df,
    source="Notebook",
    dataset_name="FLIGHTS_FULL",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="Decision_Tree_Delay_Classifier",
    description="Decision tree model trained via Notebook to identify Flight Delay application",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["scikit-learn==1.3.2"]
)

In [ ]:
y_train_df

# Push Model Input on Snowflake

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
data_df = pd.concat([X_train_df,y_train_df],axis=1,ignore_index=True)

In [ ]:
data_df.columns = ['MONTH', 'DAY', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME','DEPARTURE_DELAY','DISTANCE','SCHEDULED_ARRIVAL', 'DIVERTED', 'CANCELLED', 
          'AIR_SYSTEM_DELAY','SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY','WEATHER_DELAY','AIRLINE', 'ARRIVAL_TIME_SEGMENT','FLIGHT_DELAY']

In [ ]:
data_df.head()

In [ ]:
data_df.info()

In [ ]:
data_df['FLIGHT_DELAY'] = data_df['FLIGHT_DELAY'].astype('str')

In [ ]:
data_df.head()

In [ ]:
ins_train_sf=my_session.createDataFrame(
        data_df.values.tolist(),
        schema=data_df.columns.tolist())
ins_train_sf.write.mode("overwrite").save_as_table("TTH_DB.TTH_AIRLINE_SCHEMA.FLIGHTS_TRAINDATA")

# Push Model Output on snowflake

In [ ]:
test_df = pd.concat([X_test_df,y_test_df,y_pred_df, y_prob_df],axis=1,ignore_index=True)

In [ ]:
test_df.dropna(inplace=True)

In [ ]:
test_df.shape

In [ ]:
test_df.head()

In [ ]:
test_df.columns = ['MONTH', 'DAY', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME','DEPARTURE_DELAY','DISTANCE','SCHEDULED_ARRIVAL', 
                   'DIVERTED', 'CANCELLED', 'AIR_SYSTEM_DELAY','SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                   'WEATHER_DELAY','AIRLINE', 'ARRIVAL_TIME_SEGMENT','FLIGHT_DELAY','PREDICTED_DELAY','PROBABILITY']

In [ ]:
X1_df = test_df.head(15000)
X2_df = test_df.tail(15000)

In [ ]:
ins_train_sf=my_session.createDataFrame(
        X1_df.values.tolist(),
        schema=X1_df.columns.tolist())
ins_train_sf.write.mode("overwrite").save_as_table("TTH_DB.TTH_AIRLINE_SCHEMA.DELAY_CLASSIFIER_OUTPUT_1")

In [ ]:
ins_train_sf=my_session.createDataFrame(
        X2_df.values.tolist(),
        schema=X2_df.columns.tolist())
ins_train_sf.write.mode("overwrite").save_as_table("TTH_DB.TTH_AIRLINE_SCHEMA.DELAY_CLASSIFIER_OUTPUT_2")